In [1]:
from os import environ as ENV

DATA = ENV['PWD'] + '/data/'

In [2]:
import sys

sys.path.append('../src')

from should_be_stdlib import *
from circuit_extra import *
from circuit_postprocess import *

In [3]:
from collections import Counter
import json

import pandas as pd
import qbraid

from tqdm.notebook import tqdm

In [4]:
# number of shots to use
SHOTS = 1000

# Load circuits

In [5]:
circuit_dfs = {
    k: pd.read_excel(DATA + f'circuits_{k}.xlsx', index_col=0)
    for k in ['ang', 'amp', 'amp-qft', 'amp-ddd', 'amp-qft-ddd']
}

{k: v['qasm2'].size for k, v in circuit_dfs.items()}

{'ang': 2926,
 'amp': 2926,
 'amp-qft': 2926,
 'amp-ddd': 2926,
 'amp-qft-ddd': 2926}

In [6]:
# transpile circuits ahead of time
# if IonQ-qiskit is installed, the ionq job submission expects qiskit circuits instead of the native ionq format!!
# but qiskit transpiler is much faster so it's fine
circuits = {k: [qbraid.transpile(c, 'qiskit') for c in tqdm(v['qasm2'])] for k, v in circuit_dfs.items()}

  0%|          | 0/2926 [00:00<?, ?it/s]

/home/user/work/quadrigems/.venv/lib/python3.11/site-packages/qbraid/_entrypoints.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


  0%|          | 0/2926 [00:00<?, ?it/s]

  0%|          | 0/2926 [00:00<?, ?it/s]

  0%|          | 0/2926 [00:00<?, ?it/s]

  0%|          | 0/2926 [00:00<?, ?it/s]

In [7]:
for k, v in circuits.items():
    for i, c in enumerate(v):
        c.name = f'{k}_c{i}'

# IBMQ

## Connect to IBMQ API

In [8]:
from provider_ibmq import IBMQ

with open(ENV['HOME'] + '/work/api-keys/IBM_API_KEY') as f_api:
    ibm_api_key = f_api.readline().strip()
with open(ENV['HOME'] + '/work/api-keys/IBM_CRN') as f_crn:
    ibm_crn = f_crn.readline().strip()
ibmq = IBMQ(ibm_api_key, ibm_crn)
del ibm_api_key
del ibm_crn
ibmq.printqpus()

# selected QPU
ibm_qpu = ibmq.provider.get_device('ibm_kingston')
ibm_qpu.status()

ibmq QiskitBackend('ibm_kingston') 156qb DeviceStatus.ONLINE
ibmq QiskitBackend('ibm_fez') 156qb DeviceStatus.ONLINE
ibmq QiskitBackend('ibm_marrakesh') 156qb DeviceStatus.ONLINE
ibmq QiskitBackend('ibm_torino') 133qb DeviceStatus.ONLINE
ibmq QiskitBackend('ibm_brisbane') 127qb DeviceStatus.ONLINE
ibmq QiskitBackend('ibm_sherbrooke') 127qb DeviceStatus.ONLINE


<DeviceStatus.ONLINE: 'online'>

## Run circuits

In [9]:
# 'ALL OK' or which jobs failed?
def faileds(jobs: dict[int, any]) -> dict[int, any] | str:
    ans = {k: v.status() for k, v in jobs.items() if v.status().value != 'COMPLETED'}
    return 'ALL OK' if len(ans) == 0 else ans

### Basic test run

### Real jobs (all of them)

In [10]:
try:
    print(jobs)
except NameError:
    jobs = {}

In [11]:
# Ang
try:
    print(faileds(jobs['ang']))
except KeyError:
    if verify_run():
        jobs['ang'] = {
            i: ibm_qpu.run(circuits_chunk, shots=SHOTS)
            for i, circuits_chunk in tqdm(chunk(circuits['ang'], 50).items())
        }

KeyboardInterrupt: Interrupted by user

In [ ]:
# Amp+QFT & Amp+QFT+DDD
try:
    print(faileds(jobs['amp-qft']))
    print(faileds(jobs['amp-qft-ddd']))
except KeyError:
    if verify_run():
        jobs['amp-qft-ddd'] = {
            i: ibm_qpu.run(circuits_chunk, shots=SHOTS)
            for i, circuits_chunk in tqdm(chunk(circuits['amp-qft-ddd'], 45).items())
        }
        jobs['amp-qft'] = {
            i: ibm_qpu.run(circuits_chunk, shots=SHOTS)
            for i, circuits_chunk in tqdm(chunk(circuits['amp-qft'], 40).items())
        }

In [ ]:
# Amp & Amp+DDD
try:
    print(faileds(jobs['amp']))
    print(faileds(jobs['amp-ddd']))
except KeyError:
    if verify_run():
        jobs['amp'] = {
            i: ibm_qpu.run(circuits_chunk, shots=SHOTS)
            for i, circuits_chunk in tqdm(chunk(circuits['amp'], 47).items())
        }
        jobs['amp_ddd'] = {
            i: ibm_qpu.run(circuits_chunk, shots=SHOTS)
            for i, circuits_chunk in tqdm(chunk(circuits['amp-ddd'], 42).items())
        }

### Save/Load Job IDs
`job_ids[circuit_type][circuit_index] = job_id`

In [ ]:
# save job id list
if len(jobs) > 0:
    with open(ENV['HOME'] + '/work/jobs.json', 'wt') as f:
        json.dump(
            {
                # convert jobs to job_ids for api work
                k: [ibmq.jobid(j) for j in vs]
                for k, vs in jobs.items()
            },
            fp=f,
            indent=4,
        )

# load job id list
with open(ENV['HOME'] + '/work/jobs.json', 'rt') as f:
    job_ids = json.load(fp=f)

## Results

### Loading
From IBMQ API

In [ ]:
results = {
    k: sum([list(ibmq.jobresult(v).values()) for v in tqdm(vs, desc=k)], [])
    for k, vs in tqdm(job_ids.items(), desc='retrieve results')
}

### Calculate!
For the Ang circuits, evaluate the swap expectation.   
For the Amp-based circuits, calculate the % of 0.

In [ ]:
metrics = {
    'ang': np.array([swap_expectation({int(k, 2): v for k, v in r.items()})[1] for r in results['ang']]),
    'amp-qft': np.array([{int(k, 2): v for k, v in r.items()}.get(0, 0) for r in results['amp-qft']]) / SHOTS,
    'amp-qft-ddd': np.array([{int(k, 2): v for k, v in r.items()}.get(0, 0) for r in results['amp-qft-ddd']]) / SHOTS,
    'amp': np.array([{int(k, 2): v for k, v in r.items()}.get(0, 0) for r in results['amp']]) / SHOTS,
    'amp-ddd': np.array([{int(k, 2): v for k, v in r.items()}.get(0, 0) for r in results['amp-ddd']]) / SHOTS,
}
metrics

In [ ]:
metrics_df = pd.DataFrame({'A': circuit_dfs['ang']['A'], 'B': circuit_dfs['ang']['B'], **metrics})
metrics_df

In [ ]:
metrics_df.to_excel(DATA + 'results_ibm-kingston.xlsx')

## Circuits Analysis
Pull them back from the API

In [ ]:
circuits_real = {
    k: sum([ibmq.jobcircuit(j) for j in tqdm(js, desc=k)], [])
    for k, js in tqdm(job_ids.items(), desc='retrieve circuits')
}

### Circuit Statistics

In [ ]:
def stats(circuit_map):
    return {
        k: pd.concat(
            [
                circuit_dfs[k][['A', 'B']],
                pd.DataFrame(
                    [
                        (
                            len(v),
                            v.depth(),
                            v.num_nonlocal_gates(),
                            Counter(g.name for g in v),
                        )
                        for v in vs
                    ],
                    columns=['gates', 'depth', '2-gates', 'Gates'],
                ),
            ],
            axis=1,
        )
        for k, vs in circuit_map.items()
    }

In [ ]:
# circuits_stats = stats(circuits)
circuits_real_stats = stats(circuits_real)

In [ ]:
for k, v in circuits_real_stats.items():
    v.to_csv(DATA + f'results_ibm-kingston_stats_{k}.csv')

### Draw some

In [ ]:
n = 'ang'
print(
    circuits_real[n][0].depth(),
    circuits_real[n][0].num_nonlocal_gates(),
)
circuits_real[n][0].trim().draw('mpl', fold=60)

In [ ]:
n = 'amp'
print(
    circuits_real[n][0].depth(),
    circuits_real[n][0].num_nonlocal_gates(),
)
circuits_real[n][0].trim().draw('mpl', fold=60)

In [ ]:
n = 'amp-qft'
print(
    circuits_real[n][0].depth(),
    circuits_real[n][0].num_nonlocal_gates(),
)
circuits_real[n][0].trim().draw('mpl', fold=60)

In [ ]:
n = 'amp-qft-ddd'
print(
    circuits_real[n][0].depth(),
    circuits_real[n][0].num_nonlocal_gates(),
)
circuits_real[n][0].trim().draw('mpl', fold=60)

In [ ]:
n = 'amp-ddd'
print(
    circuits_real[n][0].depth(),
    circuits_real[n][0].num_nonlocal_gates(),
)
circuits_real[n][0].trim().draw('mpl', fold=60)